# Workflow
![Color segmentation workflow](img_embed/w8_color_segmentation.webp)

# Step 0: Setup

In [1]:
from utils import display_image, display_images, gamma_correction
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from skimage.exposure import is_low_contrast

# Step 1: load the image

In [3]:
img = cv.imread("images/w8/apples/apple1.jpg")
display_image("image", img)

# Step 2: Parameters settings

In [24]:
# copy the original image
img_copy = img.copy()
# resize and gamma correction parameters
max_width = 400
gamma = 0.7
# color segmentation parameters
red1_low = (0, 75, 100)
red1_high = (15, 255, 255)
red2_low = (170, 75, 100)
red2_high = (179, 255, 255)
green_low = (35, 50, 55)
green_high = (80, 255, 255)
# minimum area
min_area = 200

# Step 3: Implementation of red and green apples detection pipeline
![Color segmentation workflow](img_embed/color_segmentation_diagram.png)

## 3.1 Low contrast image detection
Check whether the image is of low contrast with `skimage.exposure.is_low_contrast`. If the image is low contrast, then apply gamma correction with user-defined **gamma**. 

## 3.2 Color-based segmentation
Since the aim of this project is to detect red and green apples, we will make use of HSV color space and define the suitable pixel value ranges for all the channels. Most importantly, the hue channel ranges to detect red and green objects:
* Red1: **0 - 15**
* Red2: **170 - 179**
* Green: **36 - 71**

## 3.3 Morphological operation
Apply morphological closing with user defined **structuring element** to join all the potential gaps of the segmentation mask.

## 3.4 Contour detection and post-processing
Call `cv.findContours()` and apply minimum filter on all the detected contours.

In [17]:
if img.shape[1] > max_width:
    f = max_width / img.shape[1]
    img = cv.resize(img, None, fx=f, fy=f, interpolation=cv.INTER_CUBIC)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
if is_low_contrast(gray):
    img = gamma_corretion(img, gamma=gamma)

img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# red and green mask
mask_red1 = cv.inRange(img_hsv, red1_low, red1_high)
mask_red2 = cv.inRange(img_hsv, red2_low, red2_high)
mask_green = cv.inRange(img_hsv, green_low, green_high)

mask = mask_red1 | mask_red2 | mask_green
# perform morphological closing
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
mask_final = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)

seg_res = cv.bitwise_and(img_copy, img_copy, mask=mask_final)
# for the sake of debugging, you can uncomment the below line of code
# display_images([img_copy, seg_res], ("original", "color segmentation"))

# contours
mask_red_final = mask_red1 | mask_red2
contour_red, _ = cv.findContours(mask_red_final, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contour_green, _ = cv.findContours(mask_green, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# filter contours
val_cnt_red = [cnt for cnt in contour_red if cv.contourArea(cnt) > min_area]
val_cnt_green = [cnt for cnt in contour_green if cv.contourArea(cnt) > min_area]

# drawing and annotation tools
cv.putText(seg_res, f"# red apples: {len(val_cnt_red)}",
          (25, img.shape[0] - 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
cv.putText(seg_res, f"# green apples: {len(val_cnt_green)}",
          (25, img.shape[0] - 35), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))

display_images([img_copy, seg_res], ("original", "color segmentation"))

## Function

In [25]:
def segment_green_red(img, min_area=200, max_width=400, gamma=0.7, display=True):
    """Function to segment green and red objects
    Arguments:
    ---
    img: source image
    min_area: criterion to filter contour (default: 100)
    max_width: Resize image to a certain width and preserves the aspect ratio of original image (default: 400)
    gamma: gamma correction parameter (default: 0.7)
    display: boolean (default: True)
    
    Returns:
    ---
    segmentation results with number of green and red objects"""
    img_copy = img.copy()
    # Resize image with fixed aspect ratio if the image width is greater than a certain threshold
    if img.shape[1] > max_width:
        f = max_width / img.shape[1]
        img = cv.resize(img, None, fx=f, fy=f, interpolation=cv.INTER_CUBIC)

    # denoising
    img = cv.GaussianBlur(img, (5, 5), 0)
    # check image contrast
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    if is_low_contrast(gray):
        img = gamma_corretion(img, gamma=gamma)
    # convert from BGR to HSV
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
    # red and green mask
    mask_red1 = cv.inRange(img_hsv, red1_low, red1_high)
    mask_red2 = cv.inRange(img_hsv, red2_low, red2_high)
    mask_green = cv.inRange(img_hsv, green_low, green_high)
    # red and green mask
    mask = mask_red1 | mask_red2 | mask_green
    # perform morphological closing
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    mask_final = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # segmentation result
    seg_res = cv.bitwise_and(img_copy, img_copy, mask=mask_final)
    # for the sake of debugging, you can uncomment the below line of code
    # display_images([img_copy, seg_res], ("original", "color segmentation"))
    
    # contours
    kernel2 = np.ones((5, 5), dtype=np.uint8)
    mask_red_final = mask_red1 | mask_red2
    mask_red_final = cv.erode(mask_red_final, kernel2, iterations=2)
    mask_red_final = cv.dilate(mask_red_final, kernel2, iterations=1)
    contour_red, _ = cv.findContours(mask_red_final, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    mask_green = cv.erode(mask_green, kernel2, iterations=2)
    mask_green = cv.dilate(mask_green, kernel2, iterations=1)
    contour_green, _ = cv.findContours(mask_green, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # filter contours
    val_cnt_red = [cnt for cnt in contour_red if cv.contourArea(cnt) > min_area]
    val_cnt_green = [cnt for cnt in contour_green if cv.contourArea(cnt) > min_area]
    
    # drawing and annotation tools
    cv.putText(seg_res, f"# red apples: {len(val_cnt_red)}",
              (25, img.shape[0] - 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
    cv.putText(seg_res, f"# green apples: {len(val_cnt_green)}",
              (25, img.shape[0] - 35), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))

    if display:
        display_images([img_copy, seg_res], ("original", "color segmentation"))

    return seg_res

In [27]:
file_paths = [f"images/w8/apples/apple{i}.jpg" for i in range(1, 6)]

for i, file in enumerate(file_paths):
    img = cv.imread(cv.samples.findFile(file))
    res = segment_green_red(img)
    cv.imwrite(f"images/w8/apples/app{i}_res.jpg", res)